In [2]:
from matplotlib import pyplot as plt
from icnn import ICNN
import torch as t
import numpy as np
from utils import variable

ImportError: cannot import name 'pylot'

In [ ]:
class ICNN(t.nn.Module):
    """
    CONCAVE Q network
    """
    def __init__(self, n_layers, hidden_dim, activation=SELU()):
        super(ICNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        self.activation = activation
        for l in range(n_layers - 1):
            setattr(self, 'u' + str(l), t.nn.Linear(hidden_dim, hidden_dim))

        for l in range(n_layers):
            output_dim = (l < n_layers - 1) * hidden_dim + (l == n_layers - 1) * 1
            setattr(self, 'z_u' + str(l), t.nn.Linear(hidden_dim, output_dim))
            setattr(self, 'z_au' + str(l), t.nn.Linear(hidden_dim, 2))
            setattr(self, 'z_au_' + str(l), t.nn.Linear(2, output_dim, bias=False))
            if l > 0:
                setattr(self, 'z_zu' + str(l), t.nn.Linear(hidden_dim, hidden_dim))
                setattr(self, 'z_zu_' + str(l), t.nn.Linear(hidden_dim, output_dim, bias=False))

    def forward(self, s, a):
        u = s
        for l in range(self.n_layers):
            if l < self.n_layers - 1:
                fc = getattr(self, 'u' + str(l))
                u = self.activation(fc(u))
            if l == 0:
                fc_u = getattr(self, 'z_u' + str(l))
                fc_au_ = getattr(self, 'z_au_' + str(l))
                fc_au = getattr(self, 'z_au' + str(l))
                z = self.activation(fc_u(u) + fc_au_(fc_au(u) * a))
            else:
                fc_u = getattr(self, 'z_u' + str(l))
                fc_au_ = getattr(self, 'z_au_' + str(l))
                fc_au = getattr(self, 'z_au' + str(l))
                fc_zu_ = getattr(self, 'z_zu_' + str(l))
                fc_zu = getattr(self, 'z_zu' + str(l))
                z = fc_u(u) + fc_au_(fc_au(u) * a) + fc_zu_(ReLU()(fc_zu(u)) * z)
                if l < self.n_layers - 1:
                    z = self.activation(z)
        return -z

    def make_cvx(self):
        """Make the neural network convex by absoluvaluing its W_zu weights"""
        for l in range(1, self.n_layers):
            self._modules['z_zu_' + str(l)].weight = t.nn.Parameter(data=t.abs(self._modules['z_zu_' + str(l)].weight.data))

    def proj(self):
        """If some weights became positive, set them to 0"""
        for l in range(1, self.n_layers):
            self._modules['z_zu_' + str(l)].weight = t.nn.Parameter(data=.5 * (t.abs(self._modules['z_zu_' + str(l)].weight.data) + self._modules['z_zu_' + str(l)].weight.data))


In [3]:
plt.rcParams['figure.figsize'] = (15,6)
fig, axes = plt.subplots(4,10)
icnn = ICNN(4,50)
icnn.make_cvx()
for i in range(4):
    for j in range(10):
        state = variable(np.concatenate(100*[np.random.normal(size=(1,50))], 0)).float()
        actions = variable(np.array([[a,0.] for a in np.arange(-100,100,2.)])).float()
        Qvalues = icnn.forward(state, actions).data.numpy()
        axes[i,j].plot(Qvalues)
        axes[i,j].get_xaxis().set_visible(False)
        axes[i,j].get_yaxis().set_visible(False)
plt.show()


NameError: name 'plt' is not defined